In [1]:
import sklearn
import pandas as pd
import numpy as np
import boto3
import pprint
import os
import time


import re

import sagemaker
from sagemaker import get_execution_role

In [2]:
sess = sagemaker.Session()

In [3]:
role = get_execution_role()
print(role)

arn:aws:iam::750253866451:role/sage_role


In [4]:
bucket_name = 'saurav-ml-sagemaker'

training_folder = r'ChurnModel/training/'
test_folder = r'ChurnModel/test/'
validation_folder = r'ChurnModel/validation/'

In [5]:
s3_model_output_location = r's3://{}/ChurnModel/model'.format(bucket_name)
s3_training_file_location = r's3://{0}/{1}'.format(bucket_name,training_folder)
s3_test_file_location = r's3://{0}/{1}'.format(bucket_name,test_folder)
s3_validation_file_location = r's3://{0}/{1}'.format(bucket_name, validation_folder)

In [6]:
print(s3_model_output_location)
print(s3_training_file_location)
print(s3_validation_file_location)
print(s3_test_file_location)


s3://saurav-ml-sagemaker/ChurnModel/model
s3://saurav-ml-sagemaker/ChurnModel/training/
s3://saurav-ml-sagemaker/ChurnModel/validation/
s3://saurav-ml-sagemaker/ChurnModel/test/


In [7]:
from sagemaker.amazon.amazon_estimator import get_image_uri
container = get_image_uri(boto3.Session().region_name, 'linear-learner', 'latest')

In [8]:
linear_job = "Demo-linear" + time.strftime("%Y-%m-%d-%H-%M-%S", time.gmtime())
print('Job Name is {}'.format(linear_job))

Job Name is Demo-linear2020-03-29-08-41-56


In [9]:
linear_training_params = {
    "RoleArn": role,
    "TrainingJobName": linear_job,
    "AlgorithmSpecification": {
        "TrainingImage": container,
        "TrainingInputMode": "File"
    },
    "ResourceConfig": {
        "InstanceCount": 1,
        "InstanceType": "ml.c4.2xlarge",
        "VolumeSizeInGB": 10
    },
    "InputDataConfig": [
        {
            "ChannelName": "train",
            "DataSource": {
                "S3DataSource": {
                    "S3DataType": "S3Prefix",
                    "S3Uri": s3_training_file_location,
                    "S3DataDistributionType": "ShardedByS3Key"
                }
            },
            "ContentType": "text/csv",
            "CompressionType": "None",
            "RecordWrapperType": "None"
        },
        {
            "ChannelName": "validation",
            "DataSource": {
                "S3DataSource": {
                    "S3DataType": "S3Prefix",
                    "S3Uri": s3_validation_file_location,
                    "S3DataDistributionType": "FullyReplicated"
                }
            },
            "ContentType": "text/csv",
            "CompressionType": "None",
            "RecordWrapperType": "None"
        }

    ],
    "OutputDataConfig": {
        "S3OutputPath": s3_model_output_location
    },
    "HyperParameters": {
                            "feature_dim": "13",
                            "mini_batch_size": "100",
                            "predictor_type": "binary_classifier",
                            "epochs": "10",
                            "num_models": "32",
                            "loss": "absolute_loss"
        
    },
    "StoppingCondition": {
        "MaxRuntimeInSeconds": 60 * 60
    }
}

In [10]:
region = boto3.Session().region_name
sm = boto3.client('sagemaker')

sm.create_training_job(**linear_training_params)

status = sm.describe_training_job(TrainingJobName=linear_job)['TrainingJobStatus']
print(status)
sm.get_waiter('training_job_completed_or_stopped').wait(TrainingJobName=linear_job)
if status == 'Failed':
    message = sm.describe_training_job(TrainingJobName=linear_job)['FailureReason']
    print('Training failed with the following error: {}'.format(message))
    raise Exception('Training job failed')

InProgress
